## Tarea

Implementar un sistema automático reconocedor de caracteres trazados con el mouse, los caracteres son os dígitos del 0 al 9. Por cada dígito, entrenar un modelo oculto de Markov discreto del tipo izquierda-derecha utilizando varios ejemplos de entrenamiento por cada modelo (Ej 3). El clasificador deberá decidir por el carácter asociado al modelo que tenga la mayor probabilidad de haber producido la secuencia de observaciones (secuencia de pendientes cuantificadas) del carácter que se pretende reconocer. Utilizar los algoritmos que vimos en clase (Algoritmo Forward para clasificar y algoritmo Baum-Welch para entrenar), esto quiere decir, no utilizar una librería de modelos ocultos de Markov (A menos que sea HTK o que no usen modelos discretos sino semi-contínuos de Markov). El programa debe leer los movimientos del mouse y reportar el carácter reconocido en cuanto se termine el trazo de dicho carácter.



In [7]:
# TODO ajustar estos valores.


# Arreglos de observaciones para entrenar
# 0
dig0 = [10, 9, 9, 8, 7, 6, 5, 5, 3, 1, 1, 1, 15, 1, 14, 13, 13, 12]
dig00 = [9, 8, 8, 6, 5, 4, 2, 1, 15, 15, 14, 13, 13, 12]
dig000 = [9, 8, 7, 6, 5, 5, 4, 4, 3, 2, 1, 1, 1, 14, 12, 12, 12, 11, 11, 10]
# 1
dig1 = [5, 5, 5, 5, 5, 5, 5, 5]
dig01 = [5, 5, 5, 5, 5, 5, 4]
dig001 = [5, 5, 5, 5, 5, 5, 5, 5, 6]
# 2
dig2 = [15, 15, 16, 3, 6, 7, 7, 6, 6, 3, 1, 1, 1]
dig02 = [14, 15, 16, 1, 3, 5, 6, 7, 7, 7, 7, 7, 6, 2, 1, 1, 1, 1]
dig002 = [14, 15, 1, 3, 4, 5, 6, 7, 7, 7, 7, 2, 2, 2]
# 3
dig3 = [1, 1, 1, 5, 7, 8, 8, 1, 1, 3, 5, 7, 8, 9]
dig03 = [1, 1, 1, 1, 8, 8, 8, 8, 2, 1, 2, 3, 6, 7, 8, 9, 9]
dig003 = [1, 1, 2, 5, 7, 8, 9, 1, 1, 1, 2, 4, 6, 7, 8, 9, 9]
# 4
dig4 = [9, 9, 9, 9, 9, 9, 16, 16, 16, 15,
        15, 15, 15, 15, 6, 5, 5, 5, 5, 5, 5, 5]
dig04 = [9, 9, 9, 9, 9, 9, 16, 16, 16, 15, 15, 15, 15, 3, 5, 5, 5, 5, 5]
dig004 = [8, 9, 9, 9, 9, 16, 1, 16, 14, 14, 14, 14, 2, 5, 5, 5, 5, 5]
# 5
dig5 = [9, 9, 9, 9, 9, 8, 4, 5, 1, 1, 1, 2, 3, 3, 4, 5, 7, 8, 9, 10, 10]
dig05 = [9, 9, 9, 8, 5, 4, 1, 1, 1, 3, 3, 4, 5, 7, 8, 9, 9]
dig005 = [9, 9, 9, 8, 5, 5, 1, 1, 1, 2, 2, 4, 5, 6, 8, 9, 10, 11, 12]
# 6
dig6 = [8, 7, 7, 6, 5, 4, 2, 1, 16, 14, 13, 10, 8, 8, 7]
dig06 = [9, 8, 7, 6, 5, 4, 4, 3, 2, 2, 1, 15, 13, 12, 10, 9, 8, 7, 6]
dig006 = [8, 8, 7, 6, 5, 5, 4, 3, 2, 1, 15, 13, 10, 8]
# 7
dig7 = [1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7]
dig07 = [1, 1, 1, 1, 1, 1, 3, 6, 6, 6, 6, 5, 6, 6, 6, 6]
dig007 = [1, 1, 1, 1, 1, 3, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6]
# 8
dig8 = [9, 8, 5, 2, 1, 2, 3, 4, 6, 9, 10, 11, 12, 13, 15, 15, 15, 11]
dig08 = [11, 9, 8, 7, 5, 4, 1, 1, 1, 2, 3, 4, 6, 7,
         9, 9, 10, 10, 12, 13, 14, 15, 16, 16, 15, 14]
dig008 = [10, 9, 9, 6, 5, 4, 2, 2, 2, 2, 3, 4, 4, 5, 7, 9,
          9, 10, 11, 12, 13, 14, 15, 16, 16, 16, 14, 13, 11, 10]
# 9
dig9 = [10, 8, 6, 5, 2, 1, 13, 13, 5, 5, 5, 5, 5, 5]
dig09 = [9, 8, 6, 5, 3, 1, 15, 13, 13, 5, 5, 5, 5, 5, 5, 5]
dig009 = [9, 9, 7, 4, 2, 16, 13, 14, 5, 5, 4, 4, 5, 5, 5]


In [8]:
# fuciones utiles
import numpy as np


def stochastic(matrix):
    return matrix/matrix.sum(axis=1)[:, None]

def smoothMatx(mat):
    smoothing = 1e-5
    mat[mat == 0] = smoothing
    return mat


In [9]:
# def forward(obs, states, start_prob, trans_prob, emit_prob):
#     # Initialize the matrix with the initial probabilities
#     alpha = [[start_prob[j] * emit_prob[j][obs[0]] for j in range(len(states))]]
#     # Iterate over the time steps
#     for t in range(1, len(obs)):
#         alpha.append([])
#         # Iterate over the hidden states
#         for j in range(len(states)):
#             # Compute the probability of transitioning to the current state
#             trans_prob_j = sum(trans_prob[i][j] * alpha[t-1][i] for i in range(len(states)))
#             # Update the probability of the current state
#             alpha[t].append(trans_prob_j * emit_prob[j][obs[t]])

#     # Return the final probabilities
#     return alpha

def forward(V, a, b, pi):
    p = 1
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, V.shape[0]):
        probability_of_observation = 0  # my code
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
            probability_of_observation += alpha[t, j]  # my code
        p = p * probability_of_observation  # my code
    print(alpha)
    return p  # changed


In [10]:
# pruebas

states = [0, 0, 0]
# pi = [1,0,0]
pi = np.array((1, 0, 0))
a = np.array([[0., 1., 0.],
              [0., 0.87499885, 0.12500115],
              [0., 0., 1.]])

b = np.array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               1.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000],
              [0.00000000e+000, 0.00000000e+000, 1.68229759e-177,
               1.24991944e-001, 2.50002302e-001, 2.50002302e-001,
               1.25001151e-001, 1.25001151e-001, 1.25001151e-001,
               0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000],
              [0.00000000e+000, 2.72725447e-001, 9.09084822e-002,
               6.69603684e-006, 3.77593086e-149, 0.00000000e+000,
               0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
               0.00000000e+000, 9.09084822e-002, 1.81816964e-001,
               2.72725447e-001, 0.00000000e+000, 9.09084822e-002,
               0.00000000e+000]])
# test = forward(dig000, states,pi,a,b)
# test

p = forward(np.array(dig00), a, smoothMatx(b), pi)  # changed

print(p)


[[1.00000000e+000 0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 1.25001151e-001 0.00000000e+000]
 [0.00000000e+000 1.36721088e-002 1.56252876e-007]
 [0.00000000e+000 1.49539871e-003 1.70918558e-008]
 [0.00000000e+000 3.27121049e-004 1.86943650e-009]
 [0.00000000e+000 7.15582943e-005 1.54406787e-153]
 [0.00000000e+000 1.05334414e-181 8.13164472e-007]
 [0.00000000e+000 9.21674915e-187 2.21770644e-007]
 [0.00000000e+000 8.06464490e-192 2.21770644e-012]
 [0.00000000e+000 7.05655502e-197 2.21770644e-017]
 [0.00000000e+000 6.17447752e-202 2.01608326e-018]
 [0.00000000e+000 5.40266073e-207 2.01608326e-023]
 [0.00000000e+000 4.72732193e-212 2.01608326e-028]
 [0.00000000e+000 4.13640125e-217 5.49837210e-029]]
2.3907682934948365e-151


In [11]:
import numpy as np

# Initialize the transition probabilities matrix, A, and the emission probabilities matrix, B.
# A = np.array([[0.7, 0.3], [0.4, 0.6]])
# B = np.array([[0.1, 0.4, 0.5], [0.6, 0.3, 0.1]])
A = a
B = smoothMatx(b)

# Initialize the initial state probabilities vector, pi, and the sequence of observations, O.
# pi = np.array([0.6, 0.4])
pi = np.array((1, 0, 0))
O = np.array(dig003)

# Initialize the forward probability matrix, alpha, as an array of zeros with the same shape as O.
T = len(O)
alpha = np.zeros((T, A.shape[0]))

# Set the first element of alpha as the element-wise product of pi and the first element of B.
alpha[0, :] = pi * B[:, O[0]]

# For each time step t, from 1 to T, do the following:
for t in range(1, T):
    # Calculate the forward probability for time t as the element-wise product of the previous forward probability and A, with the element-wise product of B and O at time t.
    forward_prob = alpha[t-1, :] @ A * B[:, O[t]]

    # Update the forward probability matrix, alpha, with the newly calculated forward probability.
    alpha[t, :] = forward_prob

# Calculate the log probability as the log of the sum of all elements in the forward probability matrix, alpha.
log_prob = np.log(np.sum(alpha[-1, :]))

print(alpha)

print("Log probability: ", log_prob)


[[1.00000000e-005 0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 1.00000000e-010 0.00000000e+000]
 [0.00000000e+000 1.47200846e-187 1.13636648e-012]
 [0.00000000e+000 3.22004392e-188 1.13636648e-017]
 [0.00000000e+000 3.52195083e-189 1.13636648e-022]
 [0.00000000e+000 3.85216413e-190 1.13636648e-027]
 [0.00000000e+000 3.37063919e-195 1.13636648e-032]
 [0.00000000e+000 2.94930541e-200 3.09916057e-033]
 [0.00000000e+000 2.58063884e-205 8.45219951e-034]
 [0.00000000e+000 2.25805602e-210 2.30512989e-034]
 [0.00000000e+000 0.00000000e+000 2.09555860e-035]
 [0.00000000e+000 0.00000000e+000 7.91268437e-184]
 [0.00000000e+000 0.00000000e+000 7.91268437e-189]
 [0.00000000e+000 0.00000000e+000 7.91268437e-194]
 [0.00000000e+000 0.00000000e+000 7.91268437e-199]
 [0.00000000e+000 0.00000000e+000 7.91268437e-204]
 [0.00000000e+000 0.00000000e+000 7.91268437e-209]]
Log probability:  -479.17181734747277


In [18]:
def log_likelihood(V, a, b, pi):
    """Algoritmo forward que regresa la logprobabilidad """
    T = V.shape[0]
    scale = np.zeros(T)  
    alpha = np.zeros((T, a.shape[0]))  
    # Define initial value of alpha prime
    alpha[0, :] = pi * b[:, V[0]]
    scale[0] = alpha[0, :].sum()
    alpha[0] /= scale[0]  # alpha hat
    # Induction steps
    for t in range(1, T):
        alpha_t_prime = alpha[t - 1].dot(a) * b[:, V[t]]
        scale[t] = alpha_t_prime.sum()
        alpha[t] = alpha_t_prime / scale[t]
    print(alpha)
    return np.log(scale).sum()


def forwardScale(obs, trans, emit, init):
    scale = []
    alpha = []
    N = len(obs)
    M = len(init)

    # Initialize alpha and scale
    alpha.append([0] * M)
    scale.append(0)
    for i in range(M):
        alpha[0][i] = init[i] * emit[i][obs[0]]
        scale[0] += alpha[0][i]

    # Scale alpha[0]
    for i in range(M):
        alpha[0][i] /= scale[0]

    # Compute alpha[t]
    for t in range(1, N):
        alpha.append([0] * M)
        scale.append(0)
        for i in range(M):
            for j in range(M):
                alpha[t][i] += alpha[t-1][j] * trans[j][i]
            alpha[t][i] *= emit[i][obs[t]]
            scale[t] += alpha[t][i]

        # Scale alpha[t]
        for i in range(M):
            alpha[t][i] /= scale[t]

    # Compute probability of observing the sequence
    prob = 0
    for i in range(M):
        prob += alpha[N-1][i]
    print(alpha)
    # return prob, alpha, scale, np.log(scale).sum()
    return np.log(scale).sum()


In [20]:
b= smoothMatx(b)
obs =  np.array([4, 4, 4, 4, 4, 4, 4])

print(log_likelihood(np.array(dig0), a, b, pi))
print(forwardScale(np.array(dig0), a, b, pi))


[[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.74998850e-01 1.25001150e-01]
 [0.00000000e+00 9.99975511e-01 2.44892453e-05]
 [0.00000000e+00 9.99988569e-01 1.14306947e-05]
 [0.00000000e+00 9.99988570e-01 1.14295008e-05]
 [0.00000000e+00 9.99994285e-01 5.71478301e-06]
 [0.00000000e+00 9.99994285e-01 5.71452177e-06]
 [0.00000000e+00 9.99992347e-01 7.65347852e-06]
 [0.00000000e+00 2.56584120e-04 9.99743416e-01]
 [0.00000000e+00 8.23396668e-09 9.99999992e-01]
 [0.00000000e+00 2.64174521e-13 1.00000000e+00]
 [0.00000000e+00 2.31152402e-13 1.00000000e+00]
 [0.00000000e+00 7.41617946e-18 1.00000000e+00]
 [0.00000000e+00 7.13811115e-22 1.00000000e+00]
 [0.00000000e+00 6.24583904e-22 1.00000000e+00]
 [0.00000000e+00 5.46510198e-22 1.00000000e+00]
 [0.00000000e+00 1.75339632e-26 1.00000000e+00]]
-92.07634443230444
[[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.87499885, 0.12500115], [0.0, 0.9999755107546624, 2.4489245337483167e-05],

### Ventana que captura los trazos y emite las observaciones.

In [ ]:
from tkinter import *
import math
import numpy as np
app = Tk()
app.geometry("400x400")


canvas = Canvas(app, bg='white')
canvas.pack(anchor='nw', fill='both', expand=1)

xcords = []
ycords = []


def get_x_and_y(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y


def draw(event):
    global lasx, lasy
    xcords.append(lasx)
    ycords.append(lasy)
    canvas.create_line((lasx, lasy, event.x, event.y),
                       fill='red',
                       width=2)
    lasx, lasy = event.x, event.y


def release(event):
    print("solto boton")
    genObser(xcords, ycords)


def genObser(xlist, ylist):
    """Se limpian los arreglos y se regresa la serie de observaciones"""
    xpurge, ypurge = [], []
    deglist = []
    obs = []
    for i in range(0, len(xlist), 20):
        xpurge.append(xlist[i])
        ypurge.append(ylist[i])

    for i in range(len(xpurge)-1):
        dy = ypurge[i]-ypurge[i+1]
        dx = xpurge[i]-xpurge[i+1]
        myradians = math.atan2(dy, dx)
        mydegrees = math.degrees(myradians)
        deglist.append(mydegrees)
        # print(mydegrees)

    for i in deglist:
        # para 0
        if i >= 168.75 and i <= 180:
            obs.append(0)
        if i <= -168.75:
            obs.append(0)
        # para 1
        if i > -168.75 and i < -146.25:
            obs.append(1)
        # 2
        if i > -146.25 and i < -123.75:
            obs.append(2)
        # 3
        if i > -123.75 and i < -101.25:
            obs.append(3)
        # 4
        if i > -101.25 and i < -78.75:
            obs.append(4)
        # 5
        if i > -78.75 and i < -56.25:
            obs.append(5)
        # 6
        if i > -56.25 and i < -33.75:
            obs.append(6)
        # 7
        if i > -33.75 and i < -11.25:
            obs.append(7)
        # 8
        if i >= 0 and i < 11.25:
            obs.append(8)
        if i < 0 and i > -11.25:
            obs.append(8)
        # 9
        if i >= 11.25 and i < 33.75:
            obs.append(9)
        # 10
        if i >= 33.75 and i < 56.25:
            obs.append(10)
        # 11
        if i >= 56.25 and i < 78.75:
            obs.append(11)
        # 12
        if i >= 78.75 and i < 101.25:
            obs.append(12)
        # 13
        if i >= 101.25 and i < 123.75:
            obs.append(13)
        # 14
        if i >= 123.75 and i < 146.25:
            obs.append(14)
        # 15
        if i >= 146.25 and i < 168.75:
            obs.append(15)
    print(f'Secuencia de observaciones {obs}')


canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)

app.mainloop()
